In [2]:
import pandas as pd
import os
import numpy as np
from numpy import column_stack
from pandas import read_csv
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from matplotlib import cm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from joblib import dump
from joblib import load
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
random_state = 9
params = {
    'max_depth': 5,
    'nthread' : 4,
    'learning_rate': 0.1,
    'n_estimators': 1000,
    'random_state':random_state,
}
seed = 7
test_size = 0.30

In [4]:
#Matriz de Confusion
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [5]:
print('Leyendo el archivo csv')
dataframe1 = pd.read_csv("D:\\jupyter\\csv\\UNSW_NB15_training-set.csv")
dataframe11 = pd.read_csv("D:\\jupyter\\csv\\UNSW_NB15_testing-set.csv")
print('Concatenar los archivos')
df = pd.concat([dataframe1, dataframe11])

Leyendo el archivo csv
Concatenar los archivos


In [6]:
df2 = df.drop(['id','label'], axis = 1)

In [7]:
df2['proto'] = df2['proto'].astype('category')
df2['service'] = df2['service'].astype('category')
df2['state'] = df2['state'].astype('category')
df2['attack_cat'] = df2['attack_cat'].astype('category')

In [8]:
cat_columns = df2.select_dtypes(['category']).columns
cat_columns

Index(['proto', 'service', 'state', 'attack_cat'], dtype='object')

In [9]:
df2[cat_columns] = df2[cat_columns].apply(lambda x: x.cat.codes)

In [10]:
df2 = df2.drop(df2[df2['attack_cat'] == 6].index)

In [57]:
prueba = df2.head(20)
df2.drop(df2.index[0:20],0,inplace=True)

In [11]:
Y = df2['attack_cat']
X = df2.drop(columns=['attack_cat'])

In [12]:
bi = Y.unique()

In [13]:
ataque = ['back','analy','fuzz','shell','recon','exploits','dos','worm','generic']

In [14]:
for i in range(len(bi)):
    df2[ataque[i]] = (df2['attack_cat'] == bi[i]).astype(int)

In [15]:
Y = df2.iloc[:,43:52]

In [16]:
A = [0,1,2,3,4,5,6,7,8]

In [17]:
backdoor = X[['proto','ct_src_ltm','dur','sload','sbytes','smean','ct_srv_src','stcpb','ct_dst_ltm','dmean','dbytes','sjit','ct_dst_src_ltm','ackdat','synack']]
analy = X[['proto','dur','sload','ct_src_ltm','sbytes','response_body_len','ct_dst_src_ltm','ct_dst_ltm','stcpb','dbytes','smean','ct_srv_src','ackdat','synack','sjit']]
fuzz = X[['sbytes','smean','sload','dur','stcpb','ackdat','ct_dst_src_ltm','ct_srv_src','dtcpb','dmean','proto','synack','dbytes','dload','tcprtt']]
shell  = X[['sbytes','smean','sload','dtcpb','ackdat','dload','stcpb','tcprtt','ct_srv_src','proto','ct_dst_src_ltm','djit','sjit','synack','ct_src_ltm']]
recon = X[['sbytes','stcpb','dtcpb','synack','smean','dur','ackdat','sload','proto','djit','tcprtt','sjit','dload','dinpkt','ct_srv_src']]
exploit = X[['sbytes','smean','dbytes','stcpb','dur','dmean','sload','dtcpb','dload','ackdat','djit','tcprtt','sjit','response_body_len','dinpkt']]
dos = X[['sbytes','dtcpb','smean','dbytes','dur','stcpb','djit','ackdat','synack','dload','dmean','sload','sjit','tcprtt','response_body_len']]
worm = X[['sbytes','smean','stcpb','dtcpb','sload','dload','synack','tcprtt','ackdat','ct_srv_src','dur','sinpkt','sttl','sjit','ct_src_ltm']]
generic = X[['sbytes','smean','sload','ct_src_ltm','dur','stcpb','dtcpb','synack','dload','ackdat','dmean','dbytes','djit','ct_dst_src_ltm','proto']]

# BACKDOOR

In [18]:
X_train, X_test, y_train, y_test = train_test_split(backdoor, Y.iloc[:, 0],
test_size=test_size, random_state=seed)
modelback = XGBClassifier(**params)
modelback.fit(X_train, y_train)
predictions = modelback.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.9874291141715786


array([[35288,    11],
       [  439,    59]], dtype=int64)

In [19]:
undersample = RandomUnderSampler(sampling_strategy='majority')

In [21]:
X_over, y_over = oversample.fit_resample(backdoor, Y.iloc[:, 0])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over,
test_size=test_size, random_state=seed)
modelback = XGBClassifier(**params)
modelback.fit(X_train, y_train)
predictions = modelback.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.8156370656370656


array([[409, 100],
       [ 91, 436]], dtype=int64)

In [18]:
from imblearn.over_sampling import SMOTE

In [25]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(backdoor, Y.iloc[:, 0])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_train_res, y_train_res,
test_size=test_size, random_state=seed)
modelback = XGBClassifier(**params)
modelback.fit(X_train, y_train)
predictions = modelback.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.9148206414671826


array([[29715,  5528],
       [  482, 34832]], dtype=int64)

# ANALY

In [27]:
X_train, X_test, y_train, y_test = train_test_split(analy, Y.iloc[:, 1],
test_size=test_size, random_state=seed)
modelanaly = XGBClassifier(**params)
modelanaly.fit(X_train, y_train)
predictions = modelanaly.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.9858368019666452


array([[35150,    30],
       [  477,   140]], dtype=int64)

In [29]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_over, y_over = undersample.fit_resample(analy, Y.iloc[:, 1])
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over,
test_size=test_size, random_state=seed)
modelback = XGBClassifier(**params)
modelback.fit(X_train, y_train)
predictions = modelback.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.8233333333333334


array([[468, 130],
       [ 82, 520]], dtype=int64)

In [30]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(analy, Y.iloc[:, 1])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_train_res, y_train_res,
test_size=test_size, random_state=seed)
modelanaly = XGBClassifier(**params)
modelanaly.fit(X_train, y_train)
predictions = modelanaly.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.91896921568906


array([[29806,  5237],
       [  467, 34883]], dtype=int64)

# Dos 

In [65]:
X_train, X_test, y_train, y_test = train_test_split(dos, Y.iloc[:, 6],
test_size=test_size, random_state=seed)
modeldos = XGBClassifier(**params)
modeldos.fit(X_train, y_train)
predictions = modeldos.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.8993491074671062


array([[32054,    66],
       [ 3537,   140]], dtype=int64)

In [73]:
Y.iloc[:,6].value_counts()

0    107057
1     12264
Name: dos, dtype: int64

In [74]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_over, y_over = undersample.fit_resample(dos, Y.iloc[:, 6])
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over,
test_size=test_size, random_state=seed)
modeldos = XGBClassifier(**params)
modeldos.fit(X_train, y_train)
predictions = modeldos.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.8442723196086425


array([[2771,  833],
       [ 313, 3442]], dtype=int64)

In [75]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(dos, Y.iloc[:, 6])

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X_train_res, y_train_res,
test_size=test_size, random_state=seed)
modeldos = XGBClassifier(**params)
modeldos.fit(X_train, y_train)
predictions = modeldos.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm

Accuracy 0.8856853740172803


array([[25769,  6323],
       [ 1020, 31123]], dtype=int64)

# Reconn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reconn, Y.iloc[:, 4],
test_size=test_size, random_state=seed)
modelrecon = XGBClassifier(**params)
modelrecon.fit(X_train, y_train)
predictions = modeldos.predict(X_test)
print('Accuracy',accuracy_score(y_test,predictions))
cm = confusion_matrix(y_test,predictions)
cm